In [21]:
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups

In [22]:
#fetching data
train_data = fetch_20newsgroups(remove=('headers','footers','quotes'))
X_df = pd.DataFrame(train_data.data,columns=['X'])
y_df = pd.DataFrame(train_data.target,columns=['y'])
y_df['label'] = y_df['y'].apply(lambda x:train_data.target_names[x])

In [23]:
#labels for each of the newsgroups
train_data.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

In [24]:
X_df.head()

,X
0,I was wondering if anyone out there could enli...
1,A fair number of brave souls who upgraded thei...
2,"well folks, my mac plus finally gave up the gh..."
3,\nDo you have Weitek's address/phone number? ...
4,"From article <C5owCB.n3p@world.std.com>, by to..."


In [25]:
y_df.head()

,y,label
0,7,rec.autos
1,4,comp.sys.mac.hardware
2,4,comp.sys.mac.hardware
3,1,comp.graphics
4,14,sci.space


In [26]:
X_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11314 entries, 0 to 11313
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   X       11314 non-null  object
dtypes: object(1)
memory usage: 88.5+ KB


In [27]:
y_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11314 entries, 0 to 11313
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   y       11314 non-null  int32 
 1   label   11314 non-null  object
dtypes: int32(1), object(1)
memory usage: 132.7+ KB


In [28]:
#sample data
X_df.iloc[1]['X']

"A fair number of brave souls who upgraded their SI clock oscillator have\nshared their experiences for this poll. Please send a brief message detailing\nyour experiences with the procedure. Top speed attained, CPU rated speed,\nadd on cards and adapters, heat sinks, hour of usage per day, floppy disk\nfunctionality with 800 and 1.4 m floppies are especially requested.\n\nI will be summarizing in the next two days, so please add to the network\nknowledge base if you have done the clock upgrade and haven't answered this\npoll. Thanks."

In [29]:
#sample class
y_df.iloc[1]

y                            4
label    comp.sys.mac.hardware
Name: 1, dtype: object

In [30]:
#tokenizing 
import nltk
from nltk.tokenize import word_tokenize

nltk.download('punkt')

X_df['X'] = X_df['X'].apply(lambda x: word_tokenize(x.lower()))
X_df.head()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ss\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,X
0,"[i, was, wondering, if, anyone, out, there, co..."
1,"[a, fair, number, of, brave, souls, who, upgra..."
2,"[well, folks, ,, my, mac, plus, finally, gave,..."
3,"[do, you, have, weitek, 's, address/phone, num..."
4,"[from, article, <, c5owcb.n3p, @, world.std.co..."


In [31]:
#filtering stopwords
from nltk.corpus import stopwords

nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
X_df['X'] = X_df['X'].apply(lambda x: [w for w in x if w not in stop_words])

X_df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ss\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,X
0,"[wondering, anyone, could, enlighten, car, saw..."
1,"[fair, number, brave, souls, upgraded, si, clo..."
2,"[well, folks, ,, mac, plus, finally, gave, gho..."
3,"[weitek, 's, address/phone, number, ?, 'd, lik..."
4,"[article, <, c5owcb.n3p, @, world.std.com, >, ..."


In [32]:
#stemming
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
X_df['X'] = X_df['X'].apply(lambda x: [stemmer.stem(w) for w in x])
X_df.head()

,X
0,"[wonder, anyon, could, enlighten, car, saw, da..."
1,"[fair, number, brave, soul, upgrad, si, clock,..."
2,"[well, folk, ,, mac, plu, final, gave, ghost, ..."
3,"[weitek, 's, address/phon, number, ?, 'd, like..."
4,"[articl, <, c5owcb.n3p, @, world.std.com, >, ,..."


In [33]:
#removing short word/characters
X_df['X'] = X_df['X'].apply(lambda x: [word for word in x if len(word)>2])

#removing short entries
for i in range(len(X_df['X'])):
  if len(X_df['X'].iloc[i])<10:
    X_df['X'].iloc[i] = None
    y_df['y'].iloc[i] = None
  
X_df.dropna(inplace=True)
y_df.dropna(inplace=True)

#concat
X_df['X'] = X_df['X'].apply(lambda x: ' '.join(x))

C:\Users\ss\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [34]:
#splitting dataset
y = y_df.drop('label',axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_df, y, test_size=0.1, random_state=69)

In [35]:
#converting words to vectors
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
train_vectors = vectorizer.fit_transform(X_train['X'].values)
test_vectors = vectorizer.transform(X_test['X'].values)
print(train_vectors.shape, test_vectors.shape)

(9209, 79192) (1024, 79192)


In [36]:
#training an svm model

from sklearn import svm

clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(train_vectors, y_train)

C:\Users\ss\anaconda3\lib\site-packages\sklearn\utils\validation.py:73: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(**kwargs)


SVC(kernel='linear')

In [37]:
from sklearn.metrics import classification_report

predicted = clf.predict(test_vectors)

print(classification_report(y_test,predicted))

              precision    recall  f1-score   support

         0.0       0.77      0.73      0.75        51
         1.0       0.78      0.76      0.77        50
         2.0       0.68      0.76      0.72        51
         3.0       0.71      0.67      0.69        60
         4.0       0.91      0.73      0.81        55
         5.0       0.77      0.90      0.83        52
         6.0       0.71      0.89      0.79        47
         7.0       0.71      0.83      0.77        59
         8.0       0.90      0.75      0.81        59
         9.0       0.95      0.93      0.94        42
        10.0       1.00      0.94      0.97        52
        11.0       0.93      0.73      0.82        59
        12.0       0.60      0.74      0.66        54
        13.0       0.77      0.84      0.80        43
        14.0       0.82      0.85      0.84        55
        15.0       0.84      0.88      0.86        60
        16.0       0.79      0.80      0.80        56
        17.0       0.96    